In [28]:
import pandas as pd
import numpy as np
import time
import re
import json
import tqdm
import glob
import requests as rq # http requests
import bs4 as bs4 # beautiful soup for parsing

# 1.0 - Data Collect

In [2]:
df = pd.read_json('./parsed_videos.json', lines=True)

In [6]:
## Get all unique links
unique_links = df.link.unique()
print("Lenght unique links:",len(unique_links))
print("Number of duplicated links", len(df)-len(unique_links))

Lenght unique links: 1780
Number of duplicated links 32


In [10]:
url = 'http://www.youtube.com{link}'

In [ ]:
for link in unique_links:
    urll = url.format(link=link)
    print(urll)
    response = rq.get(urll)
    
    link_name = re.search("v=(.*)", link).group(1)
    with open("./raw_data/video_{}.html".format(link_name),'w+') as output:
        output.write(response.text)
    time.sleep(2)

# 2.0 - Data Processing

In [34]:
with open("parsed_video_info.json","w+") as output:
    for video_file in tqdm.tqdm_notebook(sorted(glob.glob("./raw_data/video*"))):
        with open(video_file,"r+") as inp:
            page_html = inp.read() 
            parsed = bs4.BeautifulSoup(page_html,"html.parser") 
            
            class_watch = parsed.find_all(attr={"class":re.compile(r"watch")}) 
            id_watch = parsed.find_all(attr={"id":re.compile(r"watch")}) 
            channel = parsed.find_all("a",attr={"href":re.compile(r"channel")}) 
            meta = parsed.find_all("meta") 
        
            
            data = dict()
            
            for e in class_watch:
                colname ="_".join(e['class'])
                if 'clearfix' in colname: ## Jump condition to clear fix to not create one column per video
                    continue
                data[colname] = e.text.strip()
                
            for e in id_watch:
                colname = "_".join(e['id'])
                data[colname] = e.text.strip()
                
            for e in meta:
                colname = e.get('property')
                if colname is not None:
                    data[colname] = e['content']
                    
            for link_num, e in enumerate(channel):
                data['channel_link_{}'.format(link_num)] = e['href']
            
            
            output.write("{}\n".format(json.dumps(data)))
                    
                    